In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json

import sys
sys.path.insert(0, '../ToolBox')

import CrossValidation, MLP, Helper, ZScore, RandomForest

import os, sys
import cPickle

In [2]:
saved_train_data = open('pre_processed_data/train_preprocessed.pkl', 'rb')
normalized_train_data = cPickle.load(saved_train_data)

In [3]:
saved_test_data = open('pre_processed_data/test_preprocessed.pkl', 'rb')
normalized_test_data = cPickle.load(saved_test_data)

In [5]:
import yaml

with open("../ToolBox/params.json") as data_file:
    params = yaml.safe_load(data_file)
print 'json loaded'
labels = normalized_train_data[:,0]
print labels.shape
labels = np.reshape(labels, [labels.shape[0], 1])
print labels.shape
labels = np.subtract(np.array(labels), 1)

print normalized_train_data

data_dict = dict({'x':normalized_train_data[:,1:], 'y':Helper.convert_one_hot_encoding(labels,params['MLPandRF']['MLP']['output-dim'])})

print data_dict['y'].shape

mlp = MLP.init(params['MLPandRF']['MLP'])
mlp['train'](data_dict)

data_dict['x'] = mlp['predict'](data_dict['x'], 1)
data_dict['y'] = labels

print np.array(data_dict['x']).shape

test_transformed = mlp['predict'](normalized_test_data, 1)

rf = RandomForest.init(params['MLPandRF']['RF'])
model = rf['train'](data_dict)
result_dict = rf['predict_probabilities'](test_transformed, model)
result_dict

json loaded
(26729,)
(26729, 1)
[[ 4.         -0.8450405  -0.870233   -0.39585635 -0.46244225 -0.82232481]
 [ 3.          1.18361819  0.04882579 -0.39585635 -0.45789322 -0.78753173]
 [ 1.         -0.8450405  -0.870233   -0.05872967 -0.45334423 -0.75273871]
 ..., 
 [ 1.         -0.8450405  -0.870233    0.6155237   0.61567223 -0.09167068]
 [ 5.          1.18361819  0.9678846  -0.70712125 -0.45789322 -0.47439426]
 [ 5.          1.18361819  0.9678846  -0.39585635 -0.45789322  0.29105291]]
(26729, 5)


ValueError: Cannot feed value of shape (45, 6) for Tensor u'Placeholder:0', which has shape '(?, 5)'

In [ ]:
print result_dict.shape

lis = []
lis.append("ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer")

for idx, z in enumerate(result_dict):
    string = str(idx+1)
    for prob in z:
        string += ',' + str(prob)
    lis.append(string)

lis = np.array(lis)
np.savetxt('result_shelter_animal_outcomes.csv', lis, fmt='%s', delimiter=',')